#### Filtering out the decison tree folders.

In [1]:
import os
import pdb
import numpy as np
dt_rtl_files = []
dir_path = 'C:/Users/rrk307/Documents/NYU Research/Locking Decison Tree/TUM Collabration/drive-download-20230807T070834Z-001/1_locking/locking/ALL_DT_RFS'
for dt_file_name in (os.listdir(dir_path)):
    if 'dt' in dt_file_name:
        dt_rtl_files.append(dt_file_name)
print(dt_rtl_files)

dt_rtl_files_dict = {}
for dt_file_name in dt_rtl_files:
    with open(dir_path+'/'+dt_file_name+'/'+'decision.v','r') as fl:
        dt_rtl_files_dict[dt_file_name] = fl.readlines()
        fl.close()

['accelAdl_dt_d10', 'accelAdl_dt_d6', 'accelAdl_dt_d7', 'accelAdl_dt_d8', 'accelAdl_dt_d9', 'activities_dt_d10', 'activities_dt_d6', 'activities_dt_d7', 'activities_dt_d8', 'activities_dt_d9', 'wearable_dt_d10', 'wearable_dt_d6', 'wearable_dt_d7', 'wearable_dt_d8', 'wearable_dt_d9', 'wireless_dt_d10', 'wireless_dt_d6', 'wireless_dt_d7', 'wireless_dt_d8', 'wireless_dt_d9']


#### Creating keys

In [2]:
comparator_count = {}
for file_name in dt_rtl_files:
    counter = 0
    for line in dt_rtl_files_dict[file_name]:
        if ('\tassign comp_feat_' in line):
            counter= counter +1
    comparator_count[file_name] = counter
print(comparator_count)

dt_key = {}
print(f'Input the percentage of nodes you want to lock.')
locking_fraction=float(input())
for file_name in dt_rtl_files:
    locking_count = int(locking_fraction*comparator_count[file_name])
    key = np.random.randint(0,3,comparator_count[file_name])
    key[0:int(locking_count/2)] = 0
    key[int(locking_count/2):locking_count] = 1
    key[locking_count:] = 2
    np.random.shuffle(key)
    dt_key[file_name] = key

{'accelAdl_dt_d10': 191, 'accelAdl_dt_d6': 38, 'accelAdl_dt_d7': 62, 'accelAdl_dt_d8': 92, 'accelAdl_dt_d9': 141, 'activities_dt_d10': 243, 'activities_dt_d6': 47, 'activities_dt_d7': 79, 'activities_dt_d8': 116, 'activities_dt_d9': 171, 'wearable_dt_d10': 85, 'wearable_dt_d6': 21, 'wearable_dt_d7': 32, 'wearable_dt_d8': 43, 'wearable_dt_d9': 62, 'wireless_dt_d10': 125, 'wireless_dt_d6': 32, 'wireless_dt_d7': 48, 'wireless_dt_d8': 75, 'wireless_dt_d9': 106}
Input the percentage of nodes you want to lock.
0.85


#### Locking the RTL files.

In [3]:
for file_name in dt_rtl_files:
    comp_counter = 0
    for i,line in enumerate(dt_rtl_files_dict[file_name]):
        if ('\tassign comp_feat_' in line):
            if dt_key[file_name][comp_counter] == 0:
                dt_rtl_files_dict[file_name][i] = line.split(';\n')[0] + f'^ KEY[{comp_counter}]' + ';\n'
            elif dt_key[file_name][comp_counter] == 1:
                dt_rtl_files_dict[file_name][i] = line.split(';\n')[0] + f'~^ KEY[{comp_counter}]' + ';\n'
            else:
                pass
            comp_counter = comp_counter +1
            
for file_name in dt_rtl_files:
    for i,line in enumerate(dt_rtl_files_dict[file_name]):
        if ('\tassign comp_feat_' in line and 'KEY' in line):
            dt_rtl_files_dict[file_name][i] = line.split(' = ')[0] + ' = (' + line.split(' = ')[1].replace('];\n', ']);\n')

#### Defining the Key

In [4]:
COMPS_module_entered = 0;
for file_name in dt_rtl_files:
    for i,line in enumerate(dt_rtl_files_dict[file_name]):
        if ('module COMPS(') in line:
            dt_rtl_files_dict[file_name][i] = line.replace(line, line+'KEY,\n')
            COMPS_module_entered = 1
        
        if COMPS_module_entered == 1:
            if (');\n') in line:
                dt_rtl_files_dict[file_name][i] = line.replace(line, line+ f'input [{len(dt_key[file_name])-1}:0] KEY;\n')
                COMPS_module_entered = 0  
        
        if ('COMPS INST_COMP(') in line:
            dt_rtl_files_dict[file_name][i] = line.replace(line, line+ 'KEY,\n')
        
        if ('module decision(') in line:
            dt_rtl_files_dict[file_name][i] = line.replace(line, line+'KEY,\n')
            
        if ('output [' in line and '] decision;' in line):
            dt_rtl_files_dict[file_name][i] = line.replace(line, line+f'input [{len(dt_key[file_name])-1}:0] KEY;\n')
            
    dt_rtl_files_dict[file_name][0] = dt_rtl_files_dict[file_name][0] + f'//Correct KEY={[np.random.randint(0,2) if item == 2 else item for item in dt_key[file_name]]}'

#### Saving the locked RTL files.

In [5]:
isExist = os.path.exists('locked_dt_rtl')
if not isExist:
    os.makedirs('locked_dt_rtl')
    print(f'Locked RTL files will be in: locked_dt_rtl')
os.chdir('locked_dt_rtl')
for file_name in dt_rtl_files:
    with open('locked_'+file_name+'.v','w') as fl:
        fl.write(''.join(dt_rtl_files_dict[file_name]))
        fl.close()